In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import functools
import traceback
def get_ref_free_exc_info():
    "Free traceback from references to locals/globals to avoid circular reference leading to gc.collect() unable to reclaim memory"
    type, val, tb = sys.exc_info()
    traceback.clear_frames(tb)
    return (type, val, tb)

def gpu_mem_restore(func):
    "Reclaim GPU RAM if CUDA out of memory happened, or execution was interrupted"
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except:
            type, val, tb = get_ref_free_exc_info() # must!
            raise type(val).with_traceback(tb) from None
    return wrapper

In [3]:
from fastai.vision import *
from fastai.metrics import error_rate

In [4]:
path = Path('/home/ekami/workspace/cifar100/')

In [5]:
train = path/'train'
test = path/'test'

In [6]:
bs = 108

In [7]:
path.ls()

[PosixPath('/home/ekami/workspace/cifar100/test'),
 PosixPath('/home/ekami/workspace/cifar100/models'),
 PosixPath('/home/ekami/workspace/cifar100/train')]

In [8]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=train, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4, bs=bs).normalize(cifar_stats)

In [9]:
class gpu_mem_restore_ctx():
    " context manager to reclaim GPU RAM if CUDA out of memory happened, or execution was interrupted"
    def __enter__(self): return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        if not exc_val: return True
        traceback.clear_frames(exc_tb)
        raise exc_type(exc_val).with_traceback(exc_tb) from None

In [10]:
#Allow crashing
learn = create_cnn(data, models.resnet18 , metrics=accuracy)

## Resnet 18

In [11]:
int(bs*3.75)

405

In [12]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=train, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4, bs=408).normalize(cifar_stats)
learn = create_cnn(data, models.resnet18, metrics=accuracy)

In [13]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy
1,2.441905,1.621814,0.561000


In [14]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30, max_lr=1e-2)

epoch,train_loss,valid_loss,accuracy
1,1.686131,1.220904,0.651333
2,1.388124,0.996638,0.703500
3,1.216391,0.915040,0.721250
4,1.165618,0.937397,0.716500
5,1.166939,1.005142,0.696083
6,1.167172,1.036181,0.690250
7,1.155099,1.098619,0.678833
8,1.141750,1.031859,0.692500
9,1.122253,0.973910,0.700917
10,1.078584,0.987744,0.700667


## Resnet 18 (Mixed Prec)

In [15]:
bs

108

In [16]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=train, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4, bs=720).normalize(cifar_stats)

In [17]:
learn = to_fp16(create_cnn(data, models.resnet18, metrics=accuracy))

In [18]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy
1,2.870623,1.839730,0.519083


In [19]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,2.207675,1.754365,0.537417
2,2.115157,1.612026,0.565667
3,1.955724,1.431037,0.600417
4,1.766447,1.264467,0.639083
5,1.590725,1.155779,0.663833
6,1.441305,1.066461,0.682167
7,1.334857,1.007020,0.702500
8,1.246409,0.961123,0.711833
9,1.181972,0.911834,0.725500
10,1.128325,0.881794,0.732333


## Resnet 34 

In [20]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=train, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4, bs=248).normalize(cifar_stats)

In [21]:
learn = create_cnn(data, models.resnet34, metrics=accuracy)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /home/ekami/.torch/models/resnet34-333f7ec4.pth
100%|██████████| 87306240/87306240 [00:07<00:00, 11767995.32it/s]


In [22]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,4.100102,3.264899,0.285417
2,2.793747,1.978519,0.517667
3,1.910982,1.300082,0.636833
4,1.455095,0.999775,0.702500
5,1.241369,0.867122,0.738250
6,1.109905,0.796427,0.754583
7,1.048591,0.770618,0.762250
8,0.979058,0.730820,0.773833
9,0.928035,0.709470,0.780250
10,0.890598,0.684087,0.786833


## Resnet 34 (Mixed Precision)

In [23]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=train, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4, bs=496).normalize(cifar_stats)

In [24]:
learn = to_fp16(create_cnn(data, models.resnet34, metrics=accuracy))

In [25]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,4.522862,3.640730,0.200583
2,3.459070,2.374401,0.443583
3,2.469919,1.560894,0.582833
4,1.817846,1.165402,0.662000
5,1.449373,0.959060,0.714667
6,1.236690,0.863116,0.740500
7,1.113157,0.795895,0.756417
8,1.027290,0.765271,0.763417
9,0.967804,0.721662,0.774667
10,0.904266,0.705837,0.780333


## Resnet 50

In [26]:
#bs = 512

In [27]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=train, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4, bs=98).normalize(cifar_stats)

In [28]:
learn = create_cnn(data, models.resnet50, metrics=accuracy)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/ekami/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:19<00:00, 5150001.64it/s]


In [29]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,2.515773,1.886690,0.541167
2,1.696823,1.202539,0.663750
3,1.346115,0.940126,0.719500
4,1.105000,0.807197,0.751083
5,1.012399,0.757759,0.765250
6,0.941379,0.734523,0.770833
7,0.891324,0.690761,0.783500
8,0.819676,0.679745,0.790583
9,0.785965,0.657922,0.794000
10,0.737616,0.631472,0.806583


## Resnet 50 (Mixed Precision)

In [30]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=train, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4, bs=164).normalize(cifar_stats)

In [31]:
learn = to_fp16(create_cnn(data, models.resnet50, metrics=accuracy))

In [32]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,2.834446,2.072671,0.511667
2,1.883764,1.311698,0.639583
3,1.395265,0.999120,0.704250
4,1.161582,0.848280,0.743583
5,1.024427,0.772155,0.759583
6,0.940368,0.736896,0.770417
7,0.892585,0.707394,0.776083
8,0.829782,0.685450,0.790000
9,0.780883,0.662696,0.798583
10,0.755269,0.638769,0.802083


## Resnet 101

In [33]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=train, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4, bs=64).normalize(cifar_stats)

In [34]:
learn = create_cnn(data, models.resnet101, metrics=accuracy)

Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /home/ekami/.torch/models/resnet101-5d3b4d8f.pth
100%|██████████| 178728960/178728960 [00:29<00:00, 6127228.47it/s]


In [35]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,2.083457,1.451530,0.638333
2,1.321773,0.908532,0.733333
3,1.074472,0.728461,0.775750
4,0.985465,0.666169,0.793667
5,0.838859,0.618405,0.806750
6,0.803868,0.600917,0.812500
7,0.770296,0.592337,0.814750
8,0.712573,0.587856,0.818417
9,0.700638,0.557925,0.827667
10,0.656998,0.543478,0.833500


## Resnet 101 (Mixed Precision)

In [36]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=train, valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4, bs=116).normalize(cifar_stats)

In [37]:
learn = to_fp16(create_cnn(data, models.resnet101, metrics=accuracy))

In [38]:
with gpu_mem_restore_ctx():
    learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,accuracy
1,2.402573,1.641406,0.603583
2,1.470369,0.982278,0.721500
3,1.102205,0.770284,0.762917
4,0.931413,0.675970,0.792833
5,0.843089,0.631572,0.804000
6,0.773870,0.607783,0.812583
7,0.747394,0.579102,0.823000
8,0.698355,0.569311,0.822667
9,0.665380,0.569775,0.824250
10,0.601234,0.540831,0.833250
